In [4]:
from transformers import pipeline
from huggingface_hub import login
from dotenv import load_dotenv
from transformers import Conversation
import torch
import os
import gc
import json
import sys
from transformers.utils import logging


load_dotenv()
huggingface_key = os.getenv("HF_TOKEN")
login(token=huggingface_key)
logging.set_verbosity_error()

/Users/hanqixiao/anaconda3/envs/NLPEvals/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/hanqixiao/.cache/huggingface/token
Login successful


In [5]:
from transformers.utils import logging
logging.set_verbosity_error()

In [6]:
from datasets import load_dataset, load_from_disk
from datasets import Audio

# This dataset is a collection of different sounds of 5 seconds
dataset = load_dataset("ashraq/esc50",
                      split="train[0:10]")
dataset = dataset.cast_column("audio", Audio(sampling_rate=48_000))
# dataset = load_from_disk("./models/ashraq/esc50/train")

/Users/hanqixiao/anaconda3/envs/NLPEvals/lib/python3.11/site-packages/huggingface_hub-0.20.3-py3.8.egg/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.


In [7]:
audio_sample = dataset[0],
type(audio_sample[0]["audio"]["array"])
audio_sample[0]["audio"]["sampling_rate"]

48000

In [8]:
from IPython.display import Audio as AudioPlayer

AudioPlayer(audio_sample[0]["audio"]["array"],
             rate=audio_sample[0]["audio"]["sampling_rate"])

In [9]:
zero_shot_classifier = pipeline(
    task="zero-shot-audio-classification",
    model="./laion")

In [10]:
candidate_labels = [
    "Sand",
    "Scream",
    "Calm talking",
    "Dog screaming"
]

In [11]:
zero_shot_classifier(audio_sample[0]["audio"]["array"], candidate_labels=candidate_labels)

[{'score': 0.8488677740097046, 'label': 'Dog screaming'},
 {'score': 0.11714106798171997, 'label': 'Scream'},
 {'score': 0.019287554547190666, 'label': 'Sand'},
 {'score': 0.014703582040965557, 'label': 'Calm talking'}]

In [12]:
asr = pipeline(task="automatic-speech-recognition", model="distil-whisper/distil-small.en")

In [13]:
import gradio as gr
demo = gr.Blocks()

In [14]:
def transcribe_speech(filepath):
    if filepath is None:
        gr.Warning("No filepath/video entered")
        return ""
    output = asr(filepath, chunk_length_s=30, batch_size=30)
    return output["text"]

In [15]:
mic_transcribe = gr.Interface(
    fn=transcribe_speech,
    inputs=gr.Microphone(type="filepath"),
    outputs=gr.Textbox(label="Transcription", lines=3),
    allow_flagging="never"
)

In [16]:
file_transcribe = gr.Interface(
    fn=transcribe_speech,
    inputs=gr.Audio(sources=["upload"], type="filepath"),
    outputs=gr.Textbox(label="Transcription From File", lines=7),
    allow_flagging="never"
)

/Users/hanqixiao/anaconda3/envs/NLPEvals/lib/python3.11/site-packages/gradio/deprecation.py:43: UserWarning: You have unused kwarg parameters in Audio, please remove them: {'sources': ['upload']}
  warnings.warn(


In [17]:
with demo:
    gr.TabbedInterface(
        [mic_transcribe,
         file_transcribe],
         ["Transcribe Microphone",
          "Transcribe Audio File"]
    )
demo.launch(share=False)

/Users/hanqixiao/anaconda3/envs/NLPEvals/lib/python3.11/site-packages/gradio/blocks.py:722: UserWarning: api_name predict already exists, using predict_1
  warnings.warn(



Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
